In [2]:
# import data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets
from sklearn.manifold import TSNE
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

df = pd.read_csv('./data cleaning/All_clean_IQR.csv')
# df = df.drop(['face_width_w','face_length_L'],axis=1)
df

X = df.iloc[:, 4:] # Features
y = df.iloc[:,3:4] # Label
print(y)
print(X)

       Label
0       boss
1       boss
2       boss
3       boss
4       boss
...      ...
67303  sport
67304  sport
67305  sport
67306  sport
67307  sport

[67308 rows x 1 columns]
       Eye_R_S_W  Eye_R_B_W  Eye_L_S_W  Eye_L_B_W  Eye_R_H1_W  Eye_R_H2_W  \
0       0.186630   0.223305   0.182876   0.220148    0.013553    0.032178   
1       0.200132   0.244124   0.177286   0.207472    0.015152    0.038025   
2       0.179414   0.210726   0.192387   0.231409    0.019380    0.045778   
3       0.175631   0.208198   0.179902   0.219480    0.024991    0.057128   
4       0.189342   0.230542   0.180274   0.213068    0.008799    0.020458   
...          ...        ...        ...        ...         ...         ...   
67303   0.189668   0.224763   0.198537   0.235773    0.010172    0.024601   
67304   0.194105   0.231643   0.176857   0.212704    0.012763    0.030614   
67305   0.191906   0.226189   0.189833   0.226757    0.013453    0.031954   
67306   0.201010   0.241555   0.181272   0.21593

In [3]:
def corr_feature_detect(data,threshold=0.75): # 特徵值之間相關係數很高(高度相關)
    """ detect highly-correlated features of a Dataframe
    Parameters
    ----------
    data : pd.Dataframe
    threshold : threshold to identify the variable correlated
        
    Returns
    -------
    pairs of correlated variables
    """
    
    corrmat = data.corr()
    corrmat = corrmat.abs().unstack() # absolute value of corr coef
    corrmat = corrmat.sort_values(ascending=False)
    corrmat = corrmat[corrmat >= threshold]
    corrmat = corrmat[corrmat < 1] # remove the digonal(對角線 corr=1)
    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']
   
    grouped_feature_ls = []
    correlated_groups = []
    
    for feature in corrmat.feature1.unique():
        if feature not in grouped_feature_ls:
    
            # find all features correlated to a single feature
            correlated_block = corrmat[corrmat.feature1 == feature]
            grouped_feature_ls = grouped_feature_ls + list(
                correlated_block.feature2.unique()) + [feature]
    
            # append the block of features to the list
            correlated_groups.append(correlated_block)
    return correlated_groups


In [ ]:
# 1 高相關 > 低變異
# corr 0.75
corr = corr_feature_detect(X,threshold=0.75)
# print all the correlated feature groups!
a= []
for i in corr:
    print(i,'\n')
    a.append(i['feature2'])
#print(a)
d=[]
for ft in a:
    for i in ft:
        d.append(i)
#print(d)
deletlist = set(d)
deletlist =list(deletlist)
#print(deletlist)
len(deletlist) #102
X_c= X.drop(X[deletlist],axis=1)
#X_c.shape #(66024, 34)


# 低變異方差過濾 0.0005
selector = VarianceThreshold(threshold=0.0005)
X_cv = selector.fit_transform(X_c)  # 方差過濾

#保留特徵名稱
all_name = X_c.columns.values.tolist() 
select_name_index = selector.get_support(indices=True)  # 留下特徵的索引值，list格式
select_name = []
for i in select_name_index:
    select_name.append(all_name[i])

#X_cv.shape(66024, 8)
print(select_name)

# data normalization
data_nor_X = preprocessing.StandardScaler().fit_transform(X_cv)
data_nor_X

#標準化的資料轉成pandas後csv存檔
df_X = pd.DataFrame(data_nor_X)
df_X.columns = select_name
#pd.concat([df_X,y ],axis=1).to_csv('CorrVar_features.csv',index=False) 


In [ ]:
#2.高相關 > 卡方 (本次不適用 因為卡方要自己訂最後輸出特徵數)

X_c.shape #已做完高相關選的 X : X_c

#卡方 (k=20)
selector =  SelectKBest(chi2,k=20)
X_ck = selector.fit_transform(X_c,y)

#保留特徵名稱
all_name = X_c.columns.values.tolist() 
select_name_index2 = selector.get_support(indices=True)  # 留下特徵的索引值，list格式
select_name2 = []
for i in select_name_index2:
    select_name2.append(all_name[i])
select_name2 
# X_ck.shape # (54825, 20)
X_ck
# data normalization
data_nor_X = preprocessing.StandardScaler().fit_transform(X_ck)
data_nor_X

df_X2 = pd.DataFrame(data_nor_X,columns=select_name2)
df_X2
pd.concat([df_X2,y ],axis=1).to_csv('Corr_K_features.csv',index=False) 

In [ ]:
#3. 低變異> 高相關
# 低變異方差過濾 0.0005
selector = VarianceThreshold(threshold=0.0005)
X_v = selector.fit_transform(X)  
X_v
#保留特徵名稱
all_name = X.columns.values.tolist() 
select_name_index = selector.get_support(indices=True)  # 留下特徵的索引值，list格式
select_name = []
for i in select_name_index:
    select_name.append(all_name[i])
select_name
# print(X_v.shape)  #(66024,38)
X_v = pd.DataFrame(X_v,columns=select_name) #轉pd才能用相關係數判定
# print(X_v)

# corr 0.75
corr = corr_feature_detect(X_v,threshold=0.75)
# print all the correlated feature groups!
a= []
for i in corr:
    print(i,'\n')
    a.append(i['feature2'])
# print(a)
d=[]
for ft in a:
     for i in ft:
            d.append(i)
#print(d)
deletlist = set(d)
deletlist =list(deletlist)
# #print('deletlist:' , deletlist)
# keep = X_vc.columns
# keep = X_v.columns[keep] # keep col names
X_vc= X_v.drop(X_v[deletlist],axis=1)
X_vc.shape #(66024,9)

# data normalization
data_nor_X = preprocessing.StandardScaler().fit_transform(X_vc)
data_nor_X

#標準化的資料轉成pandas後csv存檔
df_X = pd.DataFrame(data_nor_X,columns=X_vc.columns)
#pd.concat([df_X,y ],axis=1).to_csv('VarCorr_features.csv',index=False) 

In [ ]:
#4. 卡方> 高相關
#卡方 k=50
selector =  SelectKBest(chi2,k=50)
X_k = selector.fit_transform(X,y)
#保留特徵名稱
all_name = X.columns.values.tolist() 
select_name_index = selector.get_support(indices=True)  # 留下特徵的索引值，list格式
select_name4 = []
for i in select_name_index:
    select_name4.append(all_name[i])
print(select_name4)

#print(X_k.shape)#(66024, 50)

X_k = pd.DataFrame(X_k,columns=select_name4) #轉pd才能用相關係數判定

# corr 0.75
corr = corr_feature_detect(X_k,threshold=0.75)
# print all the correlated feature groups!
a= []
for i in corr:
    print(i,'\n')
    a.append(i['feature2'])
#print(a)
d=[]
for ft in a:
    for i in ft:
        d.append(i)
#print(d)
deletlist = set(d)
deletlist =list(deletlist)
#print('deletlist:' , deletlist)
# keep = X_kc.columns
X_kc= X_k.drop(X_k[deletlist],axis=1)
X_kc.shape #(66024,12)

# data normalization
data_nor_X = preprocessing.StandardScaler().fit_transform(X_kc)
data_nor_X

#標準化的資料轉成pandas後csv存檔
df_X = pd.DataFrame(data_nor_X, columns=X_kc.columns)

#pd.concat([df_X,y ],axis=1).to_csv('K_Corr_features.csv',index=False) 

In [4]:
#5. 單純高相關過濾

# corr 0.75
corr = corr_feature_detect(X,threshold=0.75)
# print all the correlated feature groups!
a= []
for i in corr:
    print(i,'\n')
    a.append(i['feature2'])
#print(a)
d=[]
for ft in a:
    for i in ft:
        d.append(i)
#print(d)
deletlist = set(d)
deletlist =list(deletlist)
#print('deletlist:' , deletlist)
X_c= X.drop(X[deletlist],axis=1)
X_c.shape #(66024,34)
keep = X_c.columns
# data normalization
# data_nor_X = preprocessing.StandardScaler().fit_transform(X_c)
# data_nor_X

Stand = preprocessing.StandardScaler()
data_nor_X=Stand.fit_transform(X_c)

#標準化的資料轉成pandas後csv存檔
df_X = pd.DataFrame(data_nor_X)
df_X.columns = keep
pd.concat([df_X,y],axis=1).to_csv('only_Corr_features.csv',index=False) 

         feature1        feature2      corr
0     Eye_dis_S_L     Eye_dis_B_L  0.998830
200   Eye_dis_S_L      Nose_W_S_L  0.962291
210   Eye_dis_S_L      Eye_L_H1_L  0.957629
351   Eye_dis_S_L    Forehead_w_L  0.923808
364   Eye_dis_S_L    face_width_L  0.921114
420   Eye_dis_S_L      Nosehead_L  0.907592
457   Eye_dis_S_L      Nose_W_M_L  0.902374
587   Eye_dis_S_L   face_length_W  0.889079
704   Eye_dis_S_L   Eyebrow_dis_L  0.880560
752   Eye_dis_S_L       Eye_L_B_L  0.875394
761   Eye_dis_S_L       Eye_R_B_L  0.873893
792   Eye_dis_S_L       Eye_L_S_L  0.870992
808   Eye_dis_S_L       Eye_R_S_L  0.868755
923   Eye_dis_S_L     Lip_width_L  0.856989
942   Eye_dis_S_L     Eyebrow_L_L  0.854634
1062  Eye_dis_S_L     Eyebrow_R_L  0.838049
1089  Eye_dis_S_L  Eyebrow_R_H2_W  0.834491
1104  Eye_dis_S_L  Eyebrow_R_H1_W  0.832847
1130  Eye_dis_S_L  Eyebrow_L_H4_W  0.829863
1132  Eye_dis_S_L     face_down_W  0.829146
1147  Eye_dis_S_L  Eyebrow_L_H5_W  0.827686
1306  Eye_dis_S_L   face_middle_

In [ ]:
import pickle
with open ("Standard.pickle","wb") as f:
    pickle.dump(Stand,f,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
len(deletlist)

In [ ]:
a=X.columns.to_list()
len(a)

In [ ]:
b=[]
for i in a:
    if i in deletlist:
        b.append(False)
    else:
        b.append(True)

In [ ]:
b

In [ ]:
#單純高相關 > PCA 
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

pca = PCA(n_components=2) # ND -> 2D (Number of components to keep. if n_components is not set all components are kept:)
result = pca.fit_transform(df_X) 
print('coefficient of each compoments:\n{}'.format(pca.components_))
labelencoder = LabelEncoder()
y_data = labelencoder.fit_transform(df['Label'])
plt.scatter(result[:,0], result[:,1], c=y_data, s=25, alpha=0.4, marker='v')

In [ ]:
#單純高相關 > lDA 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import LabelEncoder
# https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html
lda = LDA(n_components=2) # ND -> 2D (Number of components to keep. if n_components is not set all components are kept:)
result = lda.fit_transform(df_X,y) 
result
lda.explained_variance_ratio_

#print('coefficient of each compoments:\n{}'.format(lda.explained_variance_))
# labelencoder = LabelEncoder()
# y_data = labelencoder.fit_transform(df['Label'])
# plt.scatter(result[:,0], result[:,1], c=y_data, s=25, alpha=0.4, marker='v')

# plt.xlabel('LDA1')
# plt.ylabel('LDA2')
# label = np.unique(y)
# plt.legend(label[:],loc='upper right')
# plt.show()


In [ ]:
import seaborn as sns
sns.scatterplot(x=result[:,0], 
                y=result[:,1], 
                hue=(df['Label']), 
                style=None, 
                size=None, 
                data=None, 
                palette=None, 
                sizes=None, 
                alpha=0.4)

In [ ]:

#T-sne
from sklearn import datasets
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, random_state=0) # n_components降為幾維
X_2d = tsne.fit_transform(df_X)

print(X_2d.shape)

# 畫圖需要將Label 轉為數字
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
y_data = labelencoder.fit_transform(df['Label'])
plt.scatter(X_2d[:,0], X_2d[:,1], c=y_data, s=25, alpha=0.1, marker='o')
plt.xlabel("tsne1") 
plt.ylabel("tsne2") 

In [ ]:
# corr+K > PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
#抽樣

df = pd.read_csv('./Corr_K_features.csv')
import random
from sklearn.utils import shuffle
df = shuffle(df)
df
filter1=df['Label']=='boss'  #158
filter2=df['Label']=='sport' #154
filter3=df['Label']=='entertainer' #165
filter4=df['Label']=='politician'#162
filter5=df['Label']=='doctor'#157
filter6=df['Label']=='ordinary_people'# 165

sub1=df[filter1].sample(n=None, frac=0.05, replace=False, weights=None, random_state=8)
sub2=df[filter2]
sub3=df[filter3].sample(n=None, frac=0.03, replace=False, weights=None, random_state=8)
sub4=df[filter4].sample(n=None, frac=0.15, replace=False, weights=None, random_state=8)
sub5=df[filter5].sample(n=None, frac=0.25, replace=False, weights=None, random_state=8)
sub6=df[filter6].sample(n=None, frac=0.003, replace=False, weights=None, random_state=8)
df=pd.concat([sub1,sub2,sub3,sub4,sub5,sub6],axis=0)
df_X2=df.iloc[:,:20]
df_y=df['Label']


pca2 = PCA(n_components=2) # ND -> 2D (Number of components to keep. if n_components is not set all components are kept:)
result = pca2.fit_transform(df_X2) 
print('coefficient of each compoments:\n{}'.format(pca.components_))
labelencoder = LabelEncoder()
y_data = labelencoder.fit_transform(df['Label'])
plt.scatter(result[:,0], result[:,1], c=y_data, s=25, alpha=0.5, marker='o')

In [ ]:
# pca= PCA(n_components=None)
# result = pca.fit_transform(df_X) 
# print('variance-ratio:',pca.explained_variance_ratio_)

In [ ]:
#單純高相關 > LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import LabelEncoder

lda = LDA(n_components=2) # ND -> 2D (Number of components to keep. if n_components is not set all components are kept:)
result = lda.fit_transform(df_X2,df_y) 
# print('coefficient of each compoments:\n{}'.format(pca.components_))
labelencoder = LabelEncoder()
y_data = labelencoder.fit_transform(df['Label'])
plt.scatter(result[:,0], result[:,1], c=y_data, s=25, alpha=0.6, marker='o')
plt.xlabel('LDA1')
plt.ylabel('LDA2')
label = np.unique(df_y)
label
plt.legend(label,loc='upper right')
plt.tight_layout()
plt.show()
label=np.unique(df_y)


In [ ]:
plt.matshow(abs(pca1.components_))
plt.yticks([0,1],['1st Comp','2nd Comp'],fontsize=10)
plt.colorbar()
plt.xticks(range(len(df_X.columns)),df_X.columns,rotation=65,ha='left')
plt.show()

In [ ]:
plt.matshow(abs(pca2.components_))
plt.yticks([0,1],['1st Comp','2nd Comp'],fontsize=10)
plt.colorbar()
plt.xticks(range(len(df_X2.columns)),df_X2.columns,rotation=65,ha='left')
plt.show()

In [ ]:
#單純高相關 > LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import LabelEncoder

lda = LDA(n_components=2) # ND -> 2D (Number of components to keep. if n_components is not set all components are kept:)
result = lda.fit_transform(df_X,y) 
lda.coef_
# print('coefficient of each compoments:\n{}'.format(lda.components_))
# labelencoder = LabelEncoder()
# y_data = labelencoder.fit_transform(df['Label'])
# plt.scatter(result[:,0], result[:,1], c=y_data, s=25, alpha=0.4, marker='v')
# plt.legend(loc='lower right')

In [ ]:
dir(lda)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

#SVM
X_train, X_test, y_train, y_test = train_test_split(df_X, y_data, test_size=0.3)

# we can change kernel to rbf, poly, linear
model = SVC(kernel='linear', C=0.3) #c =Soft cost 懲罰參數 (p.52) kernal(p.58)
model.fit(X_train, y_train)
#前面PCA已先做過標準化 這裡忽略標準化step
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
num_correct_samples = accuracy_score(y_test, y_pred, normalize=False)
con_matrix = confusion_matrix(y_test, y_pred)

print('number of correct sample: {}'.format(num_correct_samples))
print('accuracy: {}'.format(accuracy))
print('con_matrix: {}'.format(con_matrix))

In [ ]:
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(df_X, y_data)
print(clf.score(X_test, y_test))